In [59]:
import xml.etree.ElementTree as ET
tree = ET.parse('IR_datasets/cs.stackexchange.com/Posts.xml')
root = tree.getroot()

In [60]:
root.tag

'posts'

In [43]:
from bs4 import BeautifulSoup
import re
import string
from nltk.tokenize import word_tokenize

#this translator is used to remove the punctuation
translator = str.maketrans('', '', string.punctuation)

def sentencePreProcess(body):
    #this removes all the HTML tags
    clean_body = BeautifulSoup(body,"lxml").text
    #this removes all the punctuation
    clean_body = clean_body.translate(translator)
    #tokenize the given sentence
    word_tokens = word_tokenize(clean_body)
    #remove the stop words
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    #convert from list to sentence
    body = ' '.join(word for word in filtered_sentence)
    return body

In [57]:
def createCSV(root,quesFileName,ansFileName):
    with open(quesFileName,mode='w',encoding='utf-8') as ques_file, open(ansFileName,mode='w',encoding='utf-8') as ans_file:
        ques_file = csv.writer(ques_file, delimiter=',')
        ans_file = csv.writer(ans_file, delimiter=',')
        ques_file.writerow(['Id','Body','AcceptedAnswerId','Score','ViewCount','Title','Tags','AnswerCount','CommentCount','FavoriteCount'])
        ans_file.writerow(['Id', 'ParentId','Score','Body','CommentCount'])
        for row in root.findall('row'):
            post_type_id = row.get("PostTypeId")
            if(post_type_id == "1"):
                #It is a question
                post_id = row.get('Id')
                body = sentencePreProcess(row.get('Body'))
                accepted_answer_id = row.get('AcceptedAnswerId')
                score = row.get('Score')
                view_count = row.get('ViewCount')
                title = sentencePreProcess(row.get('Title'))
                tags = sentencePreProcess(row.get('Tags'))
                answer_count = row.get('AnswerCount')
                comment_count = row.get('CommentCount')
                fav_count = row.get('FavoriteCount')
                ques_file.writerow([post_id,body,accepted_answer_id,score,view_count,title,tags,answer_count,comment_count,fav_count])
            else:
                #it is an answer
                post_id = row.get('Id')
                parent_id = row.get('ParentId')
                score = row.get('Score')
                body = sentencePreProcess(row.get('Body'))
                comment_count = row.get('CommentCount')
                ans_file.writerow([post_id,parent_id,score,body,comment_count])
    
    
    

In [61]:
createCSV(root,"CS_questions.csv","CS_answers.csv")

In [62]:
# reload the pre-trained term embeddings
import numpy as np
import zipfile
with zipfile.ZipFile("hw2_data/glove.6B.50d.txt.zip","r") as zip_ref:
    zip_ref.extractall()

with open("glove.6B.50d.txt", "rb") as lines:
    model_word2vec = {line.split()[0]: np.array( list( map(float, line.split( )[1:]) ))
           for line in lines}


In [63]:
questions_df = pd.read_csv("ir_csv_files/CS_questions.csv")

In [66]:
def getEmbedding(body):
    output_vec = np.zeros(50)
    wordCount = 0
    words = body.split()
    for word in words:
        wordCount = wordCount + 1
        if word.encode() in model_word2vec:
            output_vec = output_vec + model_word2vec[word.encode()]
        else:
            output_vec = output_vec + np.zeros(50)
    return output_vec/(wordCount)

In [68]:
print("Shape of the questions dataframe is: ",questions_df.shape)

Shape of the questions dataframe is:  (29375, 10)


In [ ]:
#we need to get word embeddings of the body part of all the questions


In [69]:
#Load the embeddings
import numpy as np
myarray = np.fromfile('q_body_embeddings.dat', dtype=float)

In [70]:
print(myarray)

[1.29761006e-297 5.75045670e+169 6.52020716e+252 ... 3.75670872e-148
 5.73154172e+250 6.87932872e-085]


In [71]:
print(myarray.shape)

(9039701,)
